# SVMジェスチャー前処理＆学習

- 同じディレクトリに`microbit_data.csv`が必要
- 最後まで実行することで`model.sav`が上書き保存される

In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd

#pandasでcsv読込み
df = pd.read_csv('microbit_data.csv', header=None)

#カラム名を付与
df.columns = ['x', 'y', 'z', 'label']

#プロット
df.plot(y=df.columns[0:3])

In [ ]:
#使用したラベルを抽出
label_list = df['label'].unique()

#ラベル表示
print(label_list)

In [ ]:
#各ラベル毎に移動平均と標準偏差のデータに変換
import numpy as np

#移動平均と標準偏差で計算対象とするデータ値
n = 50

#空のndarray作成
X = np.empty((0, 6)) #mean_x, mean_y, mean_z, std_x, std_y, std_z
y = np.empty((0))    #label

#label_listの数だけ処理
for label_value in label_list: 
    df_part = df[df['label'] == label_value] #ラベル列がlabelと一致する行のみ抜粋
    df_xyz = df_part.iloc[:, 0:3] #x,y,zのデータ列のみ抜粋

    #移動平均
    df_mean = df_xyz.rolling(n).mean() #n個ずつのデータで平均
    df_mean.columns = ['mean_x', 'mean_y', 'mean_z'] #列名を変更

    #標準偏差
    df_std = df_xyz.rolling(n).std() #n個ずつのデータで標準偏差
    df_std.columns = ['std_x', 'std_y', 'std_z'] #列名を変更

    #移動平均と標準偏差を横に結合
    df_xyz = df_mean.join(df_std)

    #NaN（欠損値）データが１つでも含まれる行は削除
    df_xyz = df_xyz.dropna()

    #プロット
    df_xyz.plot(y=df_xyz.columns[0:3], title=str(label_value))
    df_xyz.plot(y=df_xyz.columns[3:6], title=str(label_value))
    
    #データをndarrayに変換してXに追加
    np_xyz = df_xyz.values
    X = np.append(X, np_xyz, axis=0)

    #ラベルをndarrayに変換してyに追加
    np_label = np.ones(len(df_xyz)) * label_value
    y = np.append(y, np_label , axis=0)

#確認表示
print(X)
print(y)

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split

#訓練データとテストデータに分ける
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

#モデル作成
model = LinearSVC()

#学習
model.fit(X_train, y_train)

#テストデータの予測
y_pred = model.predict(X_test)

#予測精度の表示
print(model.score(X_test, y_pred))

In [ ]:
#モデルの保存
import pickle
filename = 'model.sav'
pickle.dump(model, open(filename, 'wb'))